In [ ]:
"""
Подготовавливаем данные из таблицы POST  для залива в БД и дальнейшей работы модели

"""

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)
import numpy as np
import url from url
from category_encoders.count import CountEncoder
from category_encoders.one_hot import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer 
import re
import string
from sklearn.decomposition import PCA
np.random.seed(74)
from catboost import CatBoostClassifier
import nltk
nltk.download('wordnet')
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Илья\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:

post = pd.read_csv('post.csv')
user = pd.read_csv('user.csv')


In [3]:
def transform_user_with_drop_fich(user,drop_col = [],ohe_col=[], count_city = True, count_country = True, age_group = 'group', norm = True):
    """
    Функция получает на вход таблицу пост user.
    Опционально проводит дропает колонки, проводит OnehotEncoding и CounterEncoding
    а так же может раскодировать возвраст по группам 
    :drop_col:колонки для дропа
    :ohe_col: колонки для OnehotEncoding
    :count_city: и :count_country: (Bool) проводить ли CounterEncoding по колонкам city и country
    :age_group: 'group'/None кодировать ли возвраст
    :norm: (Bool) нормализовать ли данные при CounterEncoding
    """
    user_ = user.copy().drop(drop_col, axis=1) #убираю колонки дропа
    # Новый столбец с категориями возраста
    bins = [0, 18, 30, 60, 120]
    
    if 'age' not in drop_col and (age_group == 'group'):
        user_['age_group'] = pd.cut(user_['age'], bins, labels=['0-17', '18-30', '30-60', '60+'])
        user_ = user_.drop(['age'] , axis =1 )
    elif 'age' not in drop_col and (age_group == 'count'):
        counter_age = CountEncoder(cols=['age'], 
                            return_df=True,
                            normalize=True)
        user_ = counter_age.fit_transform(user_)
    elif 'age' not in drop_col and (age_group == 'none'):
        user_ = user_
    
    #учу CounterEncoder_User
    
    if ('city' not in drop_col) and (count_city == True):
        counter_user = CountEncoder(cols=['city'], 
                            return_df=True,
                            normalize=norm)
        user_ = counter_user.fit_transform(user_)
        
    if ('country' not in drop_col) and (count_country == True):
        counter_country = CountEncoder(cols=['country'], 
                            return_df=True,
                            normalize=norm)
        user_ = counter_country.fit_transform(user_)
        
    
    

    #OHE_User
    for col in ohe_col:
        one_hot = pd.get_dummies(user_[col], prefix=col, drop_first=True)
        user_ = pd.concat((user_.drop(col, axis=1), one_hot), axis=1)
    
    
    return user_

In [4]:
def transform_post(post, n_pca = 20, drop = True, ohe = True,lemma =True):
    """
    Функция получает на вход таблицу пост post.
    Опционально проводит OneHotEncoding типов постов,
    Лемматизацию текстов, Tf-Idf и накладывает на полученые вектора  PCA
    
    :n_pca: кол-во изерений в новом пространстве, если 0 не проводит pca b tf-idf
    :drop: = True/False дропать ли колонку текст
    :ohe: = True/False проводить ли  OneHotEncoding колонки topic
    :lemma: =True/False проводить ли лемматизацию
       
    """
    post_ = post.copy()
    wnl = WordNetLemmatizer()
    
    if lemma == True:
        def preprocessing(line, token=wnl):
            line = line.lower()
            line = re.sub(r"[{}]".format(string.punctuation), " ", line)
            line = line.replace('\n\n', ' ').replace('\n', ' ')
            line = ' '.join([token.lemmatize(x) for x in line.split(' ')])
            return line
    else:
        preprocessing = None

    #учу OHE_Post
    if ohe == True:
        one_hot = pd.get_dummies(post_['topic'], prefix='topic', drop_first=True)
        post_  = pd.concat((post_.drop(['topic'], axis=1), one_hot), axis=1)
        post_transformd = post_
    elif ohe == False:
        post_transformd = post_.drop(['topic'],axis =1 )
    
    if n_pca > 0 :
        #провожу tf-idf для Post
    
        tf = TfidfVectorizer(stop_words='english',
                             preprocessor=preprocessing, 
                             min_df = 5) #создаю экземпляр класса
        tf_idf_ = tf.fit_transform(post_['text'])#учу класс
        tf_idf_ = tf_idf_.toarray() - tf_idf_.mean() #центрируем данные

        list_col_pca = [f"PCA_{nn}" for nn in range(1,n_pca + 1)] 
        #ПРовожу PCA для User
        pca = PCA(n_components=n_pca,random_state = 74)
        #создаю экземплря PCA
        PCA_dataset = pca.fit_transform(tf_idf_) #провожу PCA 
        PCA_dataset = pd.DataFrame(PCA_dataset, columns=list_col_pca,index=post.index)
        #Трансформирую Post
    
        post_transformd = pd.concat((post_transformd, PCA_dataset), axis=1)
        if drop == True:
            post_transformd = post_transformd.drop(['text'],axis=1) 
            
    
    else:
        if drop == True:
            post_transformd = post_transformd.drop(['text'],axis=1)
        
    return post_transformd

In [5]:
drop_col = []
ohe_col=['os','source']#
ohe_topic = True # КОДИРУЕМ ЛИ TOPIC
count_city = True #кодируем ли счетчиком city
count_country = True #кодируем ли country
age_group = 'none' #кодируем ли ваозвраст в группы
norm = True # прводим ли нормализацию

user_transformd_= transform_user_with_drop_fich(user,
                                                drop_col = drop_col,
                                                ohe_col=ohe_col, 
                                                count_city = count_city, 
                                                count_country = count_country, 
                                                age_group = age_group, 
                                                norm = norm)


In [6]:
post_transformd_ = transform_post(post, 
                                  2, 
                                  drop = False, 
                                  ohe = True,
                                  lemma = False)


In [7]:
post_transformd_

,post_id,text,topic,topic_covid,topic_entertainment,topic_movie,topic_politics,topic_sport,topic_tech,PCA_1,PCA_2
0,1,UK economy facing major risks\n\nThe UK manufa...,business,0,0,0,0,0,0,-0.014171,0.181408
1,2,Aids and climate top Davos agenda\n\nClimate c...,business,0,0,0,0,0,0,-0.020236,0.196847
2,3,Asian quake hits European shares\n\nShares in ...,business,0,0,0,0,0,0,-0.016894,0.139923
3,4,India power shares jump on debut\n\nShares in ...,business,0,0,0,0,0,0,-0.005688,0.150133
4,5,Lacroix label bought by US firm\n\nLuxury good...,business,0,0,0,0,0,0,-0.011384,0.116462
...,...,...,...,...,...,...,...,...,...,...,...
7018,7315,"OK, I would not normally watch a Farrelly brot...",movie,0,0,1,0,0,0,-0.140394,-0.155916
7019,7316,I give this movie 2 stars purely because of it...,movie,0,0,1,0,0,0,-0.121802,-0.129093
7020,7317,I cant believe this film was allowed to be mad...,movie,0,0,1,0,0,0,-0.090928,-0.071366
7021,7318,The version I saw of this film was the Blockbu...,movie,0,0,1,0,0,0,-0.075142,-0.034659


In [10]:
engine = create_engine(url)

In [24]:
#записал эту таблицу
user_transformd_.to_sql('shestov_user_lesson_22_v2', con=engine) # записываем таблицу


205

In [18]:
#записал эту таблицу
post_transformd_.to_sql('shestov_post_lesson_22_v1.1', con=engine) # записываем таблицу

23